# Практическая работа №3

### Используемые библиотеки

In [1]:
import pandas as pd
import mlxtend.frequent_patterns as fp
import mlxtend.preprocessing as pp
from itertools import combinations

### Исходные данные

`data1.csv`:
```csv
tid,itemset
t1,ABCD
t2,ACDF
t3,ACDEG
t4,ABDF
t5,BCG
t6,DFG
t7,ABG
t8,CDFG
```

`data2.csv`:
```csv
tid,itemset
1,2 3 6 7
2,1 3 4 8 11
3,3 9 11
4,1 5 6 7
5,1 3 8 10 11
6,3 5 7 9 11
7,4 6 8 10 11
8,1 3 5 8 11
```

## Задание 1

In [2]:
frame = pd.read_csv('data1.csv').apply(lambda x: x.apply(list) if x.name == 'itemset' else x)
print(f"Исходный набор данных:")
frame

Исходный набор данных:


,tid,itemset
0,t1,"[A, B, C, D]"
1,t2,"[A, C, D, F]"
2,t3,"[A, C, D, E, G]"
3,t4,"[A, B, D, F]"
4,t5,"[B, C, G]"
5,t6,"[D, F, G]"
6,t7,"[A, B, G]"
7,t8,"[C, D, F, G]"


### Кодирование данных в виде матрицы

In [3]:
te = pp.TransactionEncoder()
frame = pd.DataFrame(te.fit_transform(frame['itemset'].tolist()), columns=te.columns_)
print(f"Матрица данных:")
frame

Матрица данных:


,A,B,C,D,E,F,G
0,True,True,True,True,False,False,False
1,True,False,True,True,False,True,False
2,True,False,True,True,True,False,True
3,True,True,False,True,False,True,False
4,False,True,True,False,False,False,True
5,False,False,False,True,False,True,True
6,True,True,False,False,False,False,True
7,False,False,True,True,False,True,True


### Перебор набора данных алгоритмом "apriori"

#### Перебор осуществляется следующим образом:
Пусть `support value` (минимальное количество раз, которое рассматриваемый набор элементов должен
встречаться в транзакциях для того, чтобы попасть в результирующий список) равен `3/8`.

Для всех элементов, встречающихся в транзакциях, берется множество, состоящее из этого элемента.
Считается, сколько раз эти множества встречаются в транзакциях:

| itemset | support |
| :-: | :-: |
| \[A\] | 5 |
| \[B\] | 4 |
| \[C\] | 5 |
| \[D\] | 6 |
| \[E\] | 1 |
| \[F\] | 4 |
| \[G\] | 5 |

Исключим множество `[E]`, так как оно встречается в транзакциях всего один раз.

Теперь аналогичным образом рассмотрим множества, состоящие из пар элементов, оставшихся после первого действия
(т.е. всех, кроме элемента `E`):

| itemset | support |
| :-: | :-: |
| \[A, B\] | 3 |
| \[A, C\] | 3 |
| \[A, D\] | 4 |
| \[A, F\] | 2 |
| \[A, G\] | 2 |
| \[B, C\] | 2 |
| \[B, D\] | 2 |
| \[B, F\] | 1 |
| \[B, G\] | 2 |
| \[C, D\] | 4 |
| \[C, F\] | 2 |
| \[C, G\] | 3 |
| \[D, F\] | 4 |
| \[D, G\] | 3 |
| \[F, G\] | 2 |

Оставим только множества `[A, B]`, `[A, C]`, `[A, D]`, `[C, D]`, `[C, G]`, `[D, F]`, `[D, G]`,
т.к. только они встречаются в транзакциях чаще двух раз.

Рассмотрим множества длины 3, которые можно составить из оставшихся после второго действия пар:

| itemset | support |
| :-: | :-: |
| \[A, С, D\] | 3 |
| \[C, D, G\] | 2 |

Оставим только множество `[A, С, D]`, т.к. только оно встречаются в транзакциях чаще двух раз.

Рассмотрение множеств длины 4 не имеет смысла, т.к. оставшихся после третьего действия множеств длины 3
не хватит для составления ни одного множетсва длины 4.

Вероятность правила следования из элемента `A` элемента `B` равна количеству попаданий множества `[A, B]`
в список транзакций, деленному на количеству попаданий множества `[A]` (предка) в список транзакций.

In [4]:
results = fp.apriori(frame, min_support=3.0/8.0, use_colnames=True)
results['length'] = results['itemsets'].apply(lambda x: len(x))
print(f"Часто встречающиеся наборы данных:")
results

Часто встречающиеся наборы данных:


,support,itemsets,length
0,0.625,(A),1
1,0.500,(B),1
2,0.625,(C),1
3,0.750,(D),1
4,0.500,(F),1
5,0.625,(G),1
6,0.375,"(B, A)",2
7,0.375,"(C, A)",2
8,0.500,"(D, A)",2
9,0.500,"(C, D)",2


### Перебор набора данных алгоритмом "FPGrowth"
#### Перебор осуществляется следующим образом:
Пусть `support value` (минимальное количество раз, которое рассматриваемый набор элементов должен
встречаться в транзакциях для того, чтобы попасть в результирующий список) равен `2/8`.

Для всех элементов, встречающихся в транзакциях, высчитывается количество вхождений, после чего элементы сортируются
в порядке убывания количества вхождений:

| itemset | support |
| :-: | :-: |
| D | 6 |
| A | 5 |
| C | 5 |
| G | 5 |
| B | 4 |
| F | 4 |
| E | 1 |

Исключим элемент `E`, так как он встречается в транзакциях всего один раз.

После этого составляется дерево, в которое записываются вхождения элементов из каждой транзакции:

1. Транзакция `[A, B, C, D]`:<br>
```
    { }
     |
   (D:1)
     |
   (A:1)
     |
   (C:1)
     |
   (B:1)
```
2. Транзакция `[A, C, D, F]`:
```
    { }
     |
   (D:2)
     |
   (A:2)
     |
   (C:2) - (F:1)
     |
   (B:1)
```
3. Транзакция `[A, C, D, G]`:
```
            { }
             |
           (D:3)
             |
           (A:3)
             |
   (G:1) - (C:3) - (F:1)
             |
           (B:1)
```
4. Транзакция `[A, B, D, F]`:
```
            { }
             |
           (D:4)
             |
           (A:4) - (B:1) - (F:1)
             |
   (G:1) - (C:3) - (F:1)
             |
           (B:1)
```
5. Транзакция `[B, C, G]`:
```
              { } - (C:1) - (G:1) - (B:1)
               |
             (D:4)
               |
             (A:4) - (B:1) - (F:1)
               |
     (G:1) - (C:3) - (F:1)
               |
             (B:1)
```
6. Транзакция `[D, F, G]`:
```
              { } - (C:1) - (G:1) - (B:1)
               |
             (D:5) - (G:1) - (F:1)
               |
             (A:4) - (B:1) - (F:1)
               |
     (G:1) - (C:3) - (F:1)
               |
             (B:1)
```
7. Транзакция `[A, B, G]`:
```
    (B:1) - (G:1) - (A:1) - { } - (C:1) - (G:1) - (B:1)
                             |
                           (D:5) - (G:1) - (F:1)
                             |
                           (A:4) - (B:1) - (F:1)
                             |
                   (G:1) - (C:3) - (F:1)
                             |
                           (B:1)
```
8. Транзакция `[C, D, F, G]`:
```
    (B:1) - (G:1) - (A:1) - { } - (C:1) - (G:1) - (B:1)
                             |
   (F:1) - (G:1) - (C:1) - (D:6) - (G:1) - (F:1)
                             |
                           (A:4) - (B:1) - (F:1)
                             |
                   (G:1) - (C:3) - (F:1)
                             |
                           (B:1)
```

...

* `F(4)`:
```
D(6) -> A(4) -> B(1) -> F(1) : DAB(1)
D(6) -> G(1) -> F(1) : DG(1)
D(6) -> C(1) -> G(1) -> F(1) : DCG(1)
D(6) -> A(4) -> C(3) -> F(1) : DAC(1)
```
Итого:

| itemset | support |
| :-: | :-: |
| DF | 4 |
| AF | 2 |
| CF | 2 |
| GF | 2 |
| DAF | 2 |
| DCF | 2 |
| DGF | 2 |
| DACF | 2 |

* `B(4)`:
```
C(1) -> G(1) -> B(1) : CG(1)
D(6) -> A(4) -> B(1) : DA(1)
D(6) -> A(4) -> C(3) -> B(1) : DAC(1)
A(1) -> G(1) -> B(1) : AG(1)
```
Итого:

| itemset | support |
| :-: | :-: |
| AB | 3 |
| CB | 2 |
| GB | 2 |
| DB | 2 |
| DAB | 2 |

* `G(5)`:
```
C(1) -> G(1) : C(1)
D(6) -> G(1) : D(1)
D(6) -> A(4) -> C(3) -> G(1) : DAC(1)
D(6) -> C(1) -> G(1) : D(1)
A(1) -> G(1) : A(1)
```
Итого:

| itemset | support |
| :-: | :-: |
| DG | 3 |
| AG | 2 |
| CG | 2 |

* `C(5)`:
```
C(1) : (1)
D(6) -> A(4) -> C(3) : DA(3)
D(6) -> C(1) : D(1)
```
Итого:

| itemset | support |
| :-: | :-: |
| DC | 4 |
| AC | 3 |
| DAC | 3 |

* `A(5)`:
```
D(6) -> A(4) : D(4)
A(1) : (1)
```
Итого:

| itemset | support |
| :-: | :-: |
| DA | 4 |

* `D(6)`:
```
D(6) : (6)
```

Множества длины 1 были рассмотрены ранее.

In [5]:
results = fp.fpgrowth(frame, min_support=2.0/8.0, use_colnames=True)
results['length'] = results['itemsets'].apply(lambda x: len(x))
results

,support,itemsets,length
0,0.750,(D),1
1,0.625,(C),1
2,0.625,(A),1
3,0.500,(B),1
4,0.500,(F),1
5,0.625,(G),1
6,0.500,"(C, D)",2
7,0.375,"(C, G)",2
8,0.250,"(C, G, D)",3
9,0.500,"(D, A)",2


## Задание 2

In [6]:
frame = pd.read_csv('data2.csv').apply(lambda x: x.apply(str.split) if x.name == 'itemset' else x)
print(f"Исходный набор данных:")
frame

Исходный набор данных:


,tid,itemset
0,1,"[2, 3, 6, 7]"
1,2,"[1, 3, 4, 8, 11]"
2,3,"[3, 9, 11]"
3,4,"[1, 5, 6, 7]"
4,5,"[1, 3, 8, 10, 11]"
5,6,"[3, 5, 7, 9, 11]"
6,7,"[4, 6, 8, 10, 11]"
7,8,"[1, 3, 5, 8, 11]"


In [7]:
field = [item for sub in [combinations(range(1, 12), i) for i in range(1, 12)] for item in sub]
print(f"Размер множества поиска наборов элементов:")
len(field)

Размер множества поиска наборов элементов:


2047

In [8]:
def process_leaves(leaves):
    res = []
    for leaf in leaves:
        leaf = int(leaf)
        if leaf in (12, 5):
            res += [leaf, 14]
        elif leaf in (7, 13, 11):
            res += [leaf, 15]
        elif leaf in (2, 3, 4):
            res += [leaf, 12, 14]
        elif leaf in (8, 9, 10):
            res += [leaf, 13, 15]
        else:
            res += [leaf]
    return list(set(res))

print(f"Набор высоких в таксометрии элементов:")
frame = frame.apply(lambda x: x.apply(process_leaves) if x.name == 'itemset' else x)
frame

Набор высоких в таксометрии элементов:


,tid,itemset
0,1,"[2, 3, 6, 7, 12, 14, 15]"
1,2,"[1, 3, 4, 8, 11, 12, 13, 14, 15]"
2,3,"[3, 9, 11, 12, 13, 14, 15]"
3,4,"[1, 5, 6, 7, 14, 15]"
4,5,"[1, 3, 8, 10, 11, 12, 13, 14, 15]"
5,6,"[3, 5, 7, 9, 11, 12, 13, 14, 15]"
6,7,"[4, 6, 8, 10, 11, 12, 13, 14, 15]"
7,8,"[1, 3, 5, 8, 11, 12, 13, 14, 15]"


In [9]:
te = pp.TransactionEncoder()
te_ary = te.fit_transform(frame['itemset'].tolist())
frame = pd.DataFrame(te_ary, columns=te.columns_)

results = fp.apriori(frame, min_support=7.0/8.0, use_colnames=True)
results['length'] = results['itemsets'].apply(lambda x: len(x))
results

,support,itemsets,length
0,0.875,(12),1
1,1.000,(14),1
2,1.000,(15),1
3,0.875,"(12, 14)",2
4,0.875,"(12, 15)",2
5,1.000,"(14, 15)",2
6,0.875,"(12, 14, 15)",3
